In [ ]:
from models.SResNet50 import *
from utils.utils import *
from utils.train import *

In [ ]:
import torch.optim as optim
import torch.nn as nn

In [ ]:
device = 'cuda'
net = Switchable_ResNet50(100)
net = net.to(device)
cudnn.benchmark = True

file_name = 'switchable_resnet50_cifar100.pt'
path = 'C://Users/Kim/Switchable-Normalizer-free-network/trained/'

learning_rate = 0.02
criterion = nn.CrossEntropyLoss(reduction='none')
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.001, nesterov=True)
lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 60, 90], gamma=0.1)

num_parameters = 0
for parameter in net.layer1[0].parameters(): # single block parameter count
  num_parameters += torch.prod(torch.tensor(parameter.shape))

In [ ]:
# get make train/test loaders
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=4)

In [ ]:
for epoch in range(200):
  train(epoch, net, params, train_loader, device, optimizer, criterion)
  test(epoch, net, params, test_loader, device, optimizer, criterion, path, file_name)
  lr_scheduler.step()

In [ ]:
writeToJson('C://Users/Kim/Switchable-Normalizer-free-network/stats/',params,'switchable_resnet_cifar100_stats.json' )